In [ ]:
# code for loading in data set from:
# https://pythonbasics.org/how-to-load-machine-learning-data-in-python/
import numpy as np
import matplotlib.pyplot as plt
import copy
filename = 'Data/CS170_Small_Data__82.txt'
raw_data = open(filename, 'rt')
data = np.loadtxt(raw_data)

In [ ]:
def MakePlot(accuracies,feature_sets):
    fig, ax = plt.subplots()
    ax.bar(range(len(feature_sets)), accuracies, width=0.5)
    plt.xticks(range(len(feature_sets)), feature_sets)
    plt.ylabel('Accuracy')
    plt.xlabel('Feature Set')
    fig.autofmt_xdate()
    if len(str(feature_sets[0])) < len(str(feature_sets[1])):
        plt.title('Forward Feature Search')
    else:
        plt.title('Backward Feature Search')
    plt.show()

def LeaveOneOutCrossValid(data, current_set, feature_to_add_or_remove, search_method):
    # Leave one out cross validation, returns the accuracy
    rows = data.shape[0]
    cols = data.shape[1]
    # zero columns
    if search_method == "forward":
        # zero out the columns that do not contain features in the current set and the feature to add
        for feature in range(1,cols):
            if (feature not in current_set) and (feature != feature_to_add_or_remove):
                data[:,feature] = 0
    elif search_method == "backward":
        # zero out the columns that contain the feature to remove and the features that are not in the current set
        for feature in range(1, cols):
            if (feature not in current_set) or (feature == feature_to_add_or_remove):
                data[:,feature] = 0  
    else:
        return 0

    NumCorrect = 0
    for i in range(rows):
        ObjToClassify = data[i,1:]
        LabelObjToClassify = data[i,0]
        NearestNeighborDistance = np.inf
        NearestNeighborLocation = np.inf
        for k in range(rows):
            if not k == i: # except self
                distance = np.sqrt(np.sum((ObjToClassify - data[k,1:]) ** 2))
                if distance < NearestNeighborDistance:
                    NearestNeighborDistance = distance
                    NearestNeighborLocation = k
                    NearestNeighborLabel = data[NearestNeighborLocation,0]
        if LabelObjToClassify == NearestNeighborLabel:
            NumCorrect = NumCorrect + 1
    
    accuracy = NumCorrect / rows
    return accuracy

def forward_feature_search(data):
    CurrentSet = set()
    BestAcc = 0
    # Deafult rate
    Data = copy.deepcopy(data)
    default_acc = LeaveOneOutCrossValid(Data, CurrentSet, None, "forward")
    accuracies = [default_acc]
    feature_sets = ['{}']

    for i in range(1, data.shape[1]):
        FeatureToAddAtThisLevel = set()
        BestSoFarAccuracy = 0
        
        for k in range(1, data.shape[1]):
            if not CurrentSet.intersection(set({k})):
                Data = copy.deepcopy(data)
                accuracy = LeaveOneOutCrossValid(Data, CurrentSet, k, "forward")

                if accuracy > BestSoFarAccuracy:
                    BestSoFarAccuracy = accuracy
                    FeatureToAddAtThisLevel = k
                    BestAcc = accuracy
        CurrentSet.add(FeatureToAddAtThisLevel) 
        print(f"On level {i} i added feature {FeatureToAddAtThisLevel} to current set")
        print(f"Current Set: {CurrentSet}")
        print(f"Accuracy: {BestAcc}")
        accuracies.append(BestAcc)
        feature_sets.append(CurrentSet.copy())
    
    MakePlot(accuracies, feature_sets)

def backward_feature_search(data):
    features = [x for x in range(1, data.shape[1])] 
    CurrentSet = set(features)
    BestAcc = 0
    # Deafult rate
    Data = copy.deepcopy(data)
    default_acc = LeaveOneOutCrossValid(Data, CurrentSet, None, "backward")
    accuracies = [default_acc]
    feature_sets = [str(CurrentSet.copy())]

    for i in range(1, data.shape[1]):

        # The logic should be, feature to remove at this level        
        FeatureToRemoveAtThisLevel = set()
        BestSoFarAccuracy = 0
        for k in range(1, data.shape[1]):
            if k in CurrentSet:
                # to create columns of zeros without affecting the original array
                Data = copy.deepcopy(data)
                accuracy = LeaveOneOutCrossValid(Data, CurrentSet, k, "backward")

                if accuracy > BestSoFarAccuracy:
                    BestSoFarAccuracy = accuracy
                    FeatureToRemoveAtThisLevel = k
                    BestAcc = accuracy
        CurrentSet.remove(FeatureToRemoveAtThisLevel) 
        print(f"On level {i} i removed feature {FeatureToRemoveAtThisLevel} from current set")
        print(f"Current Set: {CurrentSet}")
        print(f"Accuracy: {BestAcc}")
        accuracies.append(BestAcc)
        if len(CurrentSet) != 0:
            feature_sets.append(CurrentSet.copy())
        else:
            feature_sets.append('{}')
        print(feature_sets)
    print(len(accuracies),len(feature_sets))

    MakePlot(accuracies,feature_sets)

In [ ]:
filename = 'Data/CS170_Small_Data__82.txt'
Raw_data = open(filename, 'rt')
Data = np.loadtxt(Raw_data)

In [ ]:
# testing forward feature search on small data set
forward_feature_search(Data)

In [ ]:
# testing backward feature search on small data set
backward_feature_search(Data)

In [ ]:
filename = 'Data/CS170_Large_Data__2.txt'
Raw_data = open(filename, 'rt')
Data = np.loadtxt(Raw_data)

In [ ]:
# testing forward feature search on large data set
forward_feature_search(Data)

In [ ]:
# testing backward feature search on large data set1
backward_feature_search(Data)